In [95]:
from operator import attrgetter
from collections import defaultdict
import math
import numpy as np
import pandas as pd
from collections import namedtuple

In [96]:
# Algorithm
Dota2Metrics = namedtuple(
    'Dota2Metrics',
    ['part_number', 'has_winned', 'kills', 'assists', 'deaths', 'gold_per_minute', 'hero_damage']
)

Cs2Metrics = namedtuple('Cs2Metrics',
    ['part_number', 'has_winned', 'kills', 'deaths', 'assists', 'hs_percentage']
)

LolMetrics = namedtuple('LolMetrics',
    ['part_number', 'has_winned', 'kills', 'deaths', 'assists', 'cs_per_minute', 'vision_score']
)

def calc_score_dota2(v: Dota2Metrics):
    return (
        v.kills * 1.0 +
        v.assists * 0.5
        - v.deaths * 0.6 +
        v.gold_per_minute * 0.001 +
        v.hero_damage / 10000.0
    )


def calc_score_cs2(v: Cs2Metrics):
    return (
        v.kills * 2 +
        v.deaths * (-1.0) +
        v.assists * 1.5 +
        v.hs_percentage * 0.02
    )


def calc_score_lol(v: LolMetrics):
    return (
        v.kills * 2 +
        v.deaths * (-1) +
        v.assists * 1.5 +
        v.cs_per_minute * 0.01 +
        v.vision_score * 0.05
    )


def calc_total_score(m, f):
    MATCH_PART_FACTOR = 10.0
    WIN_LOSE_FACTOR = np.array([0.5, 1.0], np.float32)

    s = np.array(list(map(f, m)), np.float32)

    part_ns = np.array(
        list(map(attrgetter("part_number"), m)),
        np.float32
    )
    np.add(part_ns, 1.0, out=part_ns)
    np.multiply(part_ns, MATCH_PART_FACTOR, out=part_ns)

    wins = np.array(
        list(map(attrgetter("has_winned"), m)),
        np.int32
    )

    np.multiply(s, np.log10(part_ns) / math.log10(MATCH_PART_FACTOR), out=s)
    np.multiply(s, np.take(WIN_LOSE_FACTOR, wins), out=s)
    return np.sum(s)


In [97]:
# Dota 2 Dataset
teams_boom = {
    "Pakazs": [ Dota2Metrics(0, 0, 8, 10, 1, 710, 20577) ],
    "SLATEM$": [ Dota2Metrics(0, 0, 10, 9, 1, 706, 20927) ],
    "Sacred": [ Dota2Metrics(0, 0, 8, 9, 2, 647, 19845) ],
    "Matthew": [ Dota2Metrics(0, 0, 5, 14, 2, 381, 10670) ],
    "Mjz": [ Dota2Metrics(0, 0, 1, 18, 1, 297, 9065) ]
}

teams_mad_kings = {
    "Adrian": [
        Dota2Metrics(0, 1, 0, 3, 5, 582, 16522),
        Dota2Metrics(1, 0, 12, 7, 4, 718, 30968)
    ],
    "Mr.Jeans": [
        Dota2Metrics(0, 1, 3, 2, 5, 531, 15305),
        Dota2Metrics(1, 0, 7, 10, 5, 597, 25978)    
    ],
    "Smash": [
        Dota2Metrics(0, 1, 0, 1, 5, 441, 12945),
        Dota2Metrics(1, 0, 2, 10, 7, 437, 18744)
    ],
    "Genek": [
        Dota2Metrics(0, 1, 3, 2, 10, 247, 5573),
        Dota2Metrics(1, 0, 0, 15, 4, 275, 11037)],
    "Yadomi": [
        Dota2Metrics(0, 1, 1, 3, 7, 199, 4254),
        Dota2Metrics(1, 0, 1, 11, 7, 288, 10908)
    ]
}

team_boca_juniors = {
    "Parker": [ Dota2Metrics(0, 0, 3, 2, 3, 579, 20395) ],
    "Latotronii": [ Dota2Metrics(0, 0, 2, 4, 4, 477, 17546) ],
    "Italiano Gangstar": [ Dota2Metrics(0, 0, 4, 4, 4, 508, 18106) ],
    "Mariano": [ Dota2Metrics(0, 0, 2, 4, 4, 303, 12146) ],
    "Accel": [ Dota2Metrics(0, 0, 0, 7, 12, 241, 6556) ]
}

team_infinity = {
    "payk": [
        Dota2Metrics(0, 1, 7, 12, 2, 720, 28335),
        Dota2Metrics(1, 1, 14, 9, 0, 816, 47479)
    ],
    "Lumpy": [
        Dota2Metrics(0, 1, 7, 18, 0, 613, 22975),
        Dota2Metrics(1, 1, 5, 12, 4, 579, 28811)
    ],
    "Vitaly": [
        Dota2Metrics(0, 1, 7, 12, 0, 673, 25862),
        Dota2Metrics(1, 1, 4, 17, 2, 775, 38214)
    ],
    "MoOz": [
        Dota2Metrics(0, 1, 4, 16, 2, 375, 12601),
        Dota2Metrics(1, 1, 2, 17, 8, 310, 14417)
    ],
    "Gardick": [
        Dota2Metrics(0, 1, 1, 16, 7, 345, 10185),
        Dota2Metrics(1, 1, 2, 15, 8, 342, 15646)
    ]
}

dota_teams = {
    "BOOM Esports": teams_boom,
    "Mad Kings": teams_mad_kings,
    "BocaJuniors": team_boca_juniors,
    "Infinity": team_infinity
}


In [98]:
# Cs 2 Dataset
team_apeks = {
    "jkaem": [Cs2Metrics(0, 0, 8, 10, 1, 710)],
    "nawwk": [Cs2Metrics(0, 1, 7, 18, 0, 613)],
    "STYKO": [Cs2Metrics(1, 1, 5, 12, 4, 579)],
    "Kyxsan": [Cs2Metrics(1, 1, 14, 9, 0, 816)],
    "CacaNito": [Cs2Metrics(0, 1, 1, 3, 7, 199)]
}

team_alternate = {
    "awzek": [Cs2Metrics(0, 1, 7, 12, 0, 673)],
    "FreeZe": [Cs2Metrics(0, 0, 8, 10, 1, 710)],
    "PerX": [Cs2Metrics(0, 0, 8, 9, 2, 647)],
    "Spiidi": [Cs2Metrics(1, 0, 3, 1, 8, 350)],
    "slaxz": [Cs2Metrics(0, 0, 4, 7, 14, 610)]
}

team_big = {
    "tabseN": [Cs2Metrics(1, 0, 6, 7, 14, 591)],
    "Krimob": [Cs2Metrics(0, 0, 9, 4, 12, 491)],
    "mantuu": [Cs2Metrics(1, 0, 8, 15, 10, 871)],
    "s1n": [Cs2Metrics(0, 0, 10, 10, 10, 817)],
    "prosus": [Cs2Metrics(1, 1, 5, 12, 17, 917)]
}

team_sprout = {
    "Zyphon": [Cs2Metrics(0, 1, 8, 7, 12, 192)],
    "AZR": [Cs2Metrics(1, 1, 7, 8, 10, 497)],
    "spooke": [Cs2Metrics(1, 0, 3, 4, 9, 698)],
    "w0nderful": [Cs2Metrics(1, 1, 7, 12, 8, 976)],
    "raalz": [Cs2Metrics(0, 0, 5, 7, 10, 744)]
}

cs_teams = {
    "Apeks": team_apeks,
    "ALTERNATE": team_alternate, 
    "BIG": team_big,
    "Sprout": team_sprout    
}


In [99]:
# Computing results

def process_teams(teams, f):
    player_scores = {}
    player_teams = {}
    team_scores = defaultdict(lambda: 0.0)

    for t_name in teams:
        team = teams[t_name]
        for p_name in team:
            player_scores[p_name] = calc_total_score(team[p_name], f)
            player_teams[p_name] = t_name
            team_scores[t_name] += player_scores[p_name]
        team_scores[t_name] /= len(team)

    av_score = sum(player_scores.values()) / len(player_scores)

    schema = {
        "Команда": "str",
        "Игрок": "str",
        "Счет": 'float32',
        "Оценка": "float32"
    }
    players_df = pd.DataFrame(columns=schema).astype(schema)
    for i, p_name in enumerate(player_scores):
        players_df.loc[i] = (
            player_teams[p_name],
            p_name,
            player_scores[p_name],
            (player_scores[p_name] - av_score) / av_score
        )
    return players_df


dota_df = process_teams(dota_teams, calc_score_dota2)
cs_df = process_teams(cs_teams, calc_score_cs2)

dota_df.insert(1, "Дисциплина", "Dota 2")
cs_df.insert(1, "Дисциплина", "CS 2")

df = pd.concat([dota_df, cs_df])

display(df.sort_values('Оценка', ascending=False))

df = df.drop(columns=["Игрок"]).groupby(["Команда", "Дисциплина"]).mean()

display(df.sort_values("Оценка", ascending=False))


,Команда,Дисциплина,Игрок,Счет,Оценка
15,Infinity,Dota 2,payk,46.661358,3.120443
17,Infinity,Dota 2,Vitaly,36.940895,2.262075
16,Infinity,Dota 2,Lumpy,34.601051,2.055454
14,BIG,CS 2,prosus,54.435097,1.356645
3,Apeks,CS 2,Kyxsan,45.952381,0.989405
18,Infinity,Dota 2,MoOz,22.129986,0.954194
18,Sprout,CS 2,w0nderful,43.610527,0.888020
16,Sprout,CS 2,AZR,40.253872,0.742701
15,Sprout,CS 2,Zyphon,30.840000,0.335148
19,Infinity,Dota 2,Gardick,14.758884,0.303287


,,Счет,Оценка
Команда,Дисциплина,,
Infinity,Dota 2,31.018436,1.739091
Sprout,CS 2,30.061712,0.301454
BIG,CS 2,28.471558,0.232612
Apeks,CS 2,19.762486,-0.144428
ALTERNATE,CS 2,14.098473,-0.389638
BOOM Esports,Dota 2,6.864940,-0.393790
Mad Kings,Dota 2,5.925756,-0.476725
BocaJuniors,Dota 2,1.488290,-0.868576
